# Gradient Boosting From Scratch
There are a plethora of powerful machine learning packages now available for Python (most notably `sklearn`),
and we will use these in practice. But there's no better way to learn how a complex model _really_ works than to try to impliment it from first principles. Let's give it a go...

In [1]:
import pandas as pd
import numpy as np

In [2]:
def rmse(y_hat, y_true, normalize=False):
    '''RMSE represents the sample standard deviation of the differences between predicted and observed values'''
    y_hat = _convert_to_1D_np_array(y_hat)
    y_true = _convert_to_1D_np_array(y_true)
    normalization = y_true.size if normalize else 1
    return np.sqrt(np.sum((y_hat - y_true) ** 2) / normalization)

def sse(y_hat, y_true):
    return np.sum((y_hat - y_true) ** 2)

def mean_absolute_deviation(y_hat, y_true):
    '''mean absolute deviation is approximately 0.798 x standard deviation for normally distributed random variables'''
    y_hat = _convert_to_1D_np_array(y_hat)
    y_true = _convert_to_1D_np_array(y_true)
    return np.mean(np.abs(y_hat - y_true))

def median_absolute_deviation(y_hat, y_true):
    '''median absolute deviation is approximately 0.675 x standard deviation for normally distributed random variables'''
    y_hat = _convert_to_1D_np_array(y_hat)
    y_true = _convert_to_1D_np_array(y_true)
    return np.median(np.abs(y_hat - y_true))

def symmetric_mean_absolute_percent_error(y_hat, y_true):
    '''SMAPE is bound between 0 and 200%'''
    y_hat = _convert_to_1D_np_array(y_hat)
    y_true = _convert_to_1D_np_array(y_true)
    return np.sum(np.abs(y_hat - y_true) / ((np.abs(y_true) + np.abs(y_hat)) / 2.0)) * (100.0 / y_true.size)

def standardize(y):
    y = _convert_to_1D_np_array(y)
    return (y - np.mean(y)) / np.std(y)

def mean_absolute_scaled_error(y_hat, y_true):
    '''MASE from Hyndman and Koehler'''
    y_hat = _convert_to_1D_np_array(y_hat)
    y_true = _convert_to_1D_np_array(y_true)
    denom_sum = np.sum(abs(y_true[1:] - y_true[:-1]))
    error = abs(y_true - y_hat)
    denom = denom_sum / (float(len(y_hat)) - 1)
    return np.mean(error / denom)

def _convert_to_1D_np_array(y):
    if isinstance(y, (int, float, list)):
        return np.array(y)
    elif isinstance(y, pd.Series):
        return np.array(y)
    elif isinstance(y, pd.DataFrame):
        if y.shape[1] > 1:
            raise Exception('y must be 1-dimensional')
        else:
            return np.array(y.ix[:, 0])
    elif isinstance(y, np.ndarray):
        if y.ndim > 1:
            raise Exception('y must be 1-dimensional')
        else:
            return y
    else:
        raise Exception('y must be of type int, float, long or a 1-dimensional array-like object')


In [3]:
df = pd.read_csv('Gradient_boosting_example_data.csv', index_col='PersonID')
features = ['LikesGardening', 'PlaysVideoGames', 'LikesHats']
predictand = 'Age'
X = df[features]
y = df[predictand]

In [4]:
# decision tree depth-1 (stump) implimentation from first principles
for feature in X.columns:
    feature_means = df.groupby([feature])[predictand].mean()
    prediction_colname = 'stump_prediction_{}'.format(feature)
    residual_colname = 'stump_residual_{}'.format(feature)
    for index, value in enumerate(df[feature]):
        index += 1 # zero indexing correction
        df.loc[index, prediction_colname] = feature_means[value]
        df.loc[index, residual_colname] = df.loc[index, predictand] - df.loc[index, prediction_colname]

In [5]:
# decision tree on the residuals of the first
predictand = 'stump_residual_LikesGardening'
for feature in X.columns:
    feature_means = df.groupby([feature])[predictand].mean()
    prediction_colname = 'level_2_prediction_{}'.format(feature)
    residual_colname = 'level_2_residual_{}'.format(feature)
    for index, value in enumerate(df[feature]):
        index += 1 # zero indexing correction
        df.loc[index, prediction_colname] = feature_means[value]
        df.loc[index, residual_colname] = df.loc[index, predictand] - df.loc[index, prediction_colname]

In [6]:
df['combined_prediction'] = df.stump_prediction_LikesGardening + df.level_2_prediction_PlaysVideoGames
df['combined_residuals'] = df.Age - df.combined_prediction
# df[['combined_prediction', 'combined_residuals']]

,combined_prediction,combined_residuals
PersonID,,
1,15.683333,-2.683333
2,15.683333,-1.683333
3,15.683333,-0.683333
4,53.633333,-28.633333
5,15.683333,19.316667
6,64.333333,-15.333333
7,53.633333,14.366667
8,64.333333,6.666667
9,64.333333,8.666667


In [7]:
sse(df.combined_prediction, df.Age)
sse(df.stump_prediction_LikesGardening, df.Age)

1993.55

In [11]:
ytrue = [13, 25]
yhat1 = [35, 35]
improvement = -1
yhat2 = [item + improvement for item in yhat1]

def compute_gradient(prediction1, prediction2, target, loss_function):
    loss_function_gradient = loss_function(prediction2, target) - loss_function(prediction1, target)
    return loss_function_gradient

gradient_descent_1 = compute_gradient(yhat1[0], yhat2[0], ytrue[0], sse)
gradient_descent_2 = compute_gradient(yhat1[1], yhat2[1], ytrue[1], sse)
print(gradient_descent_1, gradient_descent_2)

gradient_descent_1 = compute_gradient(yhat1[0], yhat2[0], ytrue[0], median_absolute_deviation)
gradient_descent_2 = compute_gradient(yhat1[1], yhat2[1], ytrue[1], median_absolute_deviation)
print(gradient_descent_1, gradient_descent_2)

-43 -19
-1.0 -1.0
